## Assignment Week 11 - Biswajit Sharma

Load the MNIST data set.

In [1]:
# import modules

import numpy as np
import torch
import torch.optim as optim
from torchvision import datasets, transforms

In [2]:
import torch._dynamo
torch._dynamo.config.suppress_errors = True

In [3]:
# set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [4]:
# data precossing steps
transform = transforms.Compose(
    [transforms.ToTensor(),
    transforms.Normalize((0.1307,), (0.3081,))]
)

Display the first five images in the training data set (see section 8.1 in the Machine Learning with Python Cookbook). Compare these to the first five training labels.

In [5]:
# Load the MNIST dataset
train_dataset = datasets.MNIST("./data", train=True, download=True, transform=transform)
test_dataset = datasets.MNIST("./data", train=False, download=True, transform=transform)

Build and train a Keras CNN classifier on the MNIST training set.

In [6]:
##The book refers to Pytorch CNN classifier but assignment refers to keras, ask professor

In [7]:
# define CNN layeers
class ConvNet(torch.nn.Module):
    def __init__(self):
        super(ConvNet, self).__init__()
        self.conv1 = torch.nn.Conv2d(1, 32, kernel_size=3, padding=1)
        self.conv2 = torch.nn.Conv2d(32, 64, kernel_size=3, padding=1)
        self.dropout1 = torch.nn.Dropout(0.25)
        self.dropout2 = torch.nn.Dropout(0.5)
        self.fc1 = torch.nn.Linear(64 * 14 * 14, 128)
        self.fc2 = torch.nn.Linear(128, 10)
    def forward(self, x):
        x = torch.nn.functional.relu(self.conv1(x))
        x = torch.nn.functional.relu(self.conv2(x))
        x = torch.nn.functional.max_pool2d(self.dropout1(x), 2)
        x = torch.flatten(x, 1)
        x = torch.nn.functional.relu(self.fc1(self.dropout2(x)))
        x = self.fc2(x)
        return torch.nn.functional.log_softmax(x, dim=1)

In [8]:
#create data loaders
BATCH_SIZE = 64
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=True)

In [9]:
#initialize model
model = ConvNet().to(device)

In [10]:
#initialize the optimizer
optimizer = optim.Adam(model.parameters())

In [11]:
#compile the model
model = torch.compile(model)

In [12]:
#define training loop
model.train()
for batch_idx, (data, target) in enumerate(train_loader):
    data, target = data.to(device), target.to(device)
    optimizer.zero_grad()
    output = model(data)
    loss = torch.nn.functional.nll_loss(output, target)
    optimizer.step()

W0515 13:43:11.490000 28088 torch\_dynamo\convert_frame.py:824] WON'T CONVERT forward C:\Users\babub\AppData\Local\Temp\ipykernel_15376\707421746.py line 11 
W0515 13:43:11.490000 28088 torch\_dynamo\convert_frame.py:824] due to: 
W0515 13:43:11.490000 28088 torch\_dynamo\convert_frame.py:824] Traceback (most recent call last):
W0515 13:43:11.490000 28088 torch\_dynamo\convert_frame.py:824]   File "C:\Users\babub\Documents\python\pydata-book\Lib\site-packages\torch\_dynamo\convert_frame.py", line 786, in _convert_frame
W0515 13:43:11.490000 28088 torch\_dynamo\convert_frame.py:824]     result = inner_convert(
W0515 13:43:11.490000 28088 torch\_dynamo\convert_frame.py:824]              ^^^^^^^^^^^^^^
W0515 13:43:11.490000 28088 torch\_dynamo\convert_frame.py:824]   File "C:\Users\babub\Documents\python\pydata-book\Lib\site-packages\torch\_dynamo\convert_frame.py", line 400, in _convert_frame_assert
W0515 13:43:11.490000 28088 torch\_dynamo\convert_frame.py:824]     return _compile(
W051

In [15]:
#Define test loop
model.eval()
test_loss = 0
correct = 0
with torch.no_grad():
    for data, target in test_loader:
        data, target = data.to(device), target.to(device)
        output = model(data)

        #get index of the max log-probability
        test_loss += torch.nn.functional.nll_loss(
            output, target, reduction="sum"
        ).item() #summ the batch loss
        pred = output.argmax(dim=1, keepdim=True)
        correct += pred.eq(target.view_as(pred)).sum().item()
test_loss /= len(test_loader.dataset)


In [16]:
test_loss

2.305212445831299

In [17]:
correct

1010

In [ ]:
# view few rows
df.head()

Drop any rows with missing data.

In [ ]:
# check dimension of dataframe
df.shape

In [ ]:
#identify columns that have missing values
df.isna().sum()

In [ ]:
# drop rows if any field in a row has NaN
df = df.dropna(how="any", axis=0)

In [ ]:
# check dimension after drop
df.shape

In [ ]:
# check no missing values remain
df.isna().sum()

We notice that 134 rows have been dropped due presence of missing values.

Convert the categorical features into dummy variables.

In [ ]:
# check column datatypes
df.dtypes

In [ ]:
#using pandas get_dummies to encode categorical variables
categorical_feature_names = [cols for cols in df.columns if (df[cols].dtype == "object" and cols != "Loan_Status")]
df_with_dummies = pd.get_dummies(df, columns=categorical_feature_names, drop_first=True, dtype="int")

In [ ]:
# view few rows after applying dummies
df_with_dummies.head()

Split the data into a training and test set, where the “Loan_Status” column is the target.

In [ ]:
# define feature and target names
target = "Loan_Status"
features = [cols for cols in df_with_dummies.columns if cols != "Loan_Status"]

In [ ]:
# using sklearn's train_test_split method to split the dataset
features_train, features_test, target_train, target_test = train_test_split(
    df_with_dummies[features], df_with_dummies[target], test_size=0.2, random_state=1
)

Create a pipeline with a min-max scaler and a KNN classifier.

In [ ]:
# create a min-max scaler 
scaler = MinMaxScaler()

In [ ]:
# create a default KNN classifier
default_knn = KNeighborsClassifier()

In [ ]:
# define pipeline
pipe = Pipeline(
    [
        ("scaler",scaler),
        ("knn", default_knn)
    ]
)

Fit a default KNN classifier to the data with this pipeline. Report the model accuracy on the test set. 

In [ ]:
# fit the pipeline with training set
default_knn_model = pipe.fit(features_train, target_train)

In [ ]:
#predict loan status for test set using default KNN classifier
predicted_target_test = default_knn_model.predict(features_test)

In [ ]:
# calculate accuracy for test set using a default KNN classifier
print(f"Accuracy: {accuracy_score(target_test, predicted_target_test)}")

We see that model accuracy of the _default KNN Classifier_ is $0.73$.

Create a search space for your KNN classifier where your “n_neighbors” parameter varies from 1 to 10.

In [ ]:
#create search space 
grid_search_space = [
    {"knn__n_neighbors": [i for i in range(1,11)]}
] 

Fit a grid search with your pipeline, search space, and 5-fold cross-validation to find the best value for the “n_neighbors” parameter.

In [ ]:
# create grid search
gcv = GridSearchCV(
    pipe,
    grid_search_space,
    cv=5,
    verbose=1
)

In [ ]:
#fit the grid search
best_model = gcv.fit(features_train, target_train)

In [ ]:
# show best model parameters
best_model.best_estimator_.get_params()

In [ ]:
print(f"Best value for n_neighbors is {best_model.best_estimator_.get_params()['knn__n_neighbors']}")

We see that best model has $5$ neighbors

Find the accuracy of the grid search best model on the test set. Note: It is possible that this will not be an improvement over the default model, but likely it will be.

In [ ]:
#predict loan status for test set using the best KNN classifier
predicted_best_model_target_test = best_model.predict(features_test)

In [ ]:
# calculate accuracy for test set using best KNN classifier
print(f"Accuracy: {accuracy_score(target_test, predicted_best_model_target_test)}")

The accuracy is still $0.73$. So, there is no improvement in accuracy over the default model.  

Now, repeat steps 6 and 7 with the same pipeline, but expand your search space to include logistic regression and random forest models with the hyperparameter values in section 12.3 of the Machine Learning with Python Cookbook.

In [ ]:
# define model objects
logistic = LogisticRegression(max_iter=500, solver="liblinear")
random_forest = RandomForestClassifier()
knn = KNeighborsClassifier()

In [ ]:
# define pipeline
pipe_multiple = Pipeline(
    steps=[
        ("scaler", scaler),
        ("classifier", knn)
    ]
)   

In [ ]:
# set hyperparameters search space for all the classifiers
hyperparameters_search_space = [
    {
        "classifier": [knn],
        "classifier__n_neighbors": [i for i in range(1,11)]
    },
    {
        "classifier": [logistic],
        "classifier__penalty": ["l1", "l2"],
        "classifier__C": np.logspace(0,4,10)
    },
    {
        "classifier": [random_forest],
        "classifier__n_estimators": [10,100,1000],
        "classifier__max_features": [1,2,3]
    }
]


In [ ]:
# set up grid search for above hyperparameters
gcv_multiple = GridSearchCV(
    pipe_multiple,
    hyperparameters_search_space,
    cv=5,
    verbose=1,
    n_jobs=-1
)

In [ ]:
# fit the grid search
best_model_multiple = gcv_multiple.fit(features_train, target_train)

What are the best model and hyperparameters found in the grid search? Find the accuracy of this model on the test set.

In [ ]:
#show best model parameters
print(best_model_multiple.best_params_)
best_model_multiple.best_estimator_.get_params()

In [ ]:
#predict loan status for test set using best model 
predicted_logistic_target_test = best_model_multiple.predict(features_test)

In [ ]:
# calculate accuracy for test set a default KNN classifier
print(f"Accuracy: {accuracy_score(target_test, predicted_logistic_target_test)}")

Best model found is _Logistic Regression_ which yeilded an accuracy of $0.74$

### Summary

We observed that the accuracy of the _default_ KNN classifier is $0.73$. We used grid search approach to identify the best `n_neighbors` value to use with KNN classifier on this loan approval dataset. The grid search yeilded that $5$ neighbors generates best performce of the KNN classifier.  KNN classifier using $5$ neighbors achieved an accuracy of  $0.73$. Therefore, _no improvement_ in _accuracy_ was achieved when using _$5$ neighbors in KNN classifier over the default KNN classifier_.

Grid search was expanded to include Logistic Regression and Random Forest classifier along with KNN classifier to identify the best performing model along with its hyperparameters. The results showed that Logistic Regression classifier was identified as the best model, yeilding an accuracy of $0.74$ with hyper parameter `penalty=l1` and `C=1.0`. There is _no significant increase_ in _accuracy_ (just $1$%) for the selected Logistic Regression classifier over the default and $5$ neighbors KNN classifer models.